In [ ]:
!pip install clip_interrogator open_clip_torch gradio transformers


In [ ]:
from clip_interrogator import Config, Interrogator

config = Config()
config.clip_model_name = 'ViT-L-14/openai'
config.caption_model_name = 'blip-large'
ci = Interrogator(config)


Loading caption model blip-large...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading CLIP model ViT-L-14/openai...


/usr/local/lib/python3.11/dist-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


Loaded CLIP model and data in 21.94 seconds.


In [ ]:
# from transformers import pipeline

# # Load any conversational model (or LLM)
# llm = pipeline("text-generation", model="gpt2")  # You can use bigger models too

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:  22%|##2       | 2.82G/12.8G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:  39%|###8      | 2.84G/7.32G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def generate_response(image, mode, user_prompt):
    # Step 1: Get image caption/description using CLIP Interrogator
    ci.config.chunk_size = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    ci.config.flavor_intermediate_count = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    image = image.convert('RGB')

    if mode == 'best':
        image_description = ci.interrogate(image)
    elif mode == 'classic':
        image_description = ci.interrogate_classic(image)
    elif mode == 'fast':
        image_description = ci.interrogate_fast(image)
    elif mode == 'negative':
        image_description = ci.interrogate_negative(image)

    # Simplified prompt to LLM: treat caption as context
    prompt_to_llm = f"Image description: {image_description}\n\n{user_prompt}\n\n"

    llm_output = llm(prompt_to_llm, max_length=150, do_sample=True, top_p=0.9, temperature=0.8)[0]['generated_text']

    # Extract just the response (skip repeating the prompt)
    generated_response = llm_output[len(prompt_to_llm):].strip()

    return image_description, generated_response



In [ ]:
# def generate_response(image, mode, user_prompt):
#     # Step 1: Get image caption/description using CLIP Interrogator
#     ci.config.chunk_size = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
#     ci.config.flavor_intermediate_count = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
#     image = image.convert('RGB')

#     if mode == 'best':
#         image_description = ci.interrogate(image)
#     elif mode == 'classic':
#         image_description = ci.interrogate_classic(image)
#     elif mode == 'fast':
#         image_description = ci.interrogate_fast(image)
#     elif mode == 'negative':
#         image_description = ci.interrogate_negative(image)

#     # Step 2: Combine with user text prompt
#     combined_prompt = f"Image Description: {image_description}\nUser Prompt: {user_prompt}\nResponse:"

#     # Step 3: Generate response from LLM
#     llm_output = llm(combined_prompt, max_length=150, do_sample=True, top_p=0.9, temperature=0.5)[0]['generated_text']

#     return image_description, llm_output


In [ ]:
import gradio as gr

def gradio_ui():
    with gr.Blocks() as ui:
        with gr.Row():
            image = gr.Image(type='pil', label="Upload Image")
            mode = gr.Radio(['best', 'fast', 'classic', 'negative'], label='Image Analysis Mode', value='best')
        user_prompt = gr.Textbox(label="Your Text Prompt", placeholder="Ask something about the image...")
        image_caption = gr.Textbox(label="Image Description")
        final_response = gr.Textbox(label="Combined Response")
        generate_btn = gr.Button("Generate Response")

        generate_btn.click(generate_response, inputs=[image, mode, user_prompt], outputs=[image_caption, final_response])

    ui.launch()

gradio_ui()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc7038f73951ba7322.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
